In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from childes_mi.utils.paths import DATA_DIR, FIGURE_DIR
from childes_mi.utils.general import flatten,save_fig

In [3]:
from childes_mi.information_theory import model_fitting as mf

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.autonotebook import tqdm

/mnt/cube/tsainbur/conda_envs/tpy3/lib/python3.6/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [5]:
MI_DF = pd.read_pickle(DATA_DIR/'mi/childes_mi.pickle')

In [6]:
MI_DF[:3]

,MI,MI_var,shuff_MI,shuff_MI_var,distances,age_cohort_low,age_cohort_high,n_words,n_unique_words
0,"[6.0615547382637684, 5.953026573946322, 5.8999...","[0.06497202939114834, 0.06421181443545038, 0.0...","[5.79799893493386, 5.805771966570329, 5.792474...","[0.06474471004074468, 0.06537742978608413, 0.0...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",0.5,1,2371,72
1,"[10.373016081368098, 10.130972857078175, 9.987...","[0.00937963690006051, 0.009307961610168364, 0....","[9.60947032947951, 9.60572819037825, 9.5994570...","[0.009097935454051041, 0.009124346562127784, 0...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",1.0,1.5,67747,3609
2,"[11.349881924265677, 11.087999639009142, 10.96...","[0.005174827403727939, 0.005136394630766223, 0...","[10.681207621740015, 10.677630292519837, 10.67...","[0.005060342080401666, 0.005066649080465537, 0...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",1.5,2,232191,8134


In [7]:
MI_DF = MI_DF.assign(**{i:np.nan for i in ['exp_results', 'pow_results', 'concat_results',
     'R2_exp', 'R2_concat', 'R2_power', 'AICc_exp',
     'AICc_concat', 'AICc_power', 'bestfitmodel', 'curvature', 'min_peak']})
MI_DF['curvature'] = MI_DF['curvature'].astype(object)

In [8]:
n = 100 # max distance for computation
for idx, row in tqdm(MI_DF.iterrows(), total=len(MI_DF)):
    # get signal
    sig = np.array(row.MI-row.shuff_MI)
    distances = row.distances
    
    sig = sig
    
    # fit models
    results_power, results_exp, results_pow_exp, best_fit_model = mf.fit_models(distances, sig)
    
    # get fit results
    R2_exp, R2_concat, R2_power, AICc_exp, \
        AICc_pow, AICc_concat = mf.fit_results(sig, distances, 
                                              results_exp, results_power,
                                              results_pow_exp)
    
    
    # get model y
    distances_mod = np.logspace(0,np.log10(n), base=10, num=1000)
    if best_fit_model == 'pow_exp':
        y_model = mf.get_y(mf.pow_exp_decay, results_pow_exp, distances_mod)
    elif best_fit_model == 'exp':
        y_model = mf.get_y(mf.exp_decay, results_exp, distances_mod)
    elif best_fit_model == 'pow':
        y_model = mf.get_y(mf.powerlaw_decay, results_power, distances_mod)
    
    # get curvature of model_y
    curvature_model = mf.curvature(np.log(y_model))
    
    # if the best fit model is pow_exp, then grab the min peak
    if best_fit_model == 'pow_exp':
        # get peaks of curvature
        peaks = np.where((
            (curvature_model[:-1] < curvature_model[1:])[1:] & (curvature_model[1:] < curvature_model[:-1])[:-1]
        ))
        min_peak = peaks[0][0]
    else:
        min_peak = np.nan

    # get save model fit results to MI_DF
    MI_DF.loc[idx, np.array(['exp_results', 'pow_results', 'concat_results',
                         'R2_exp', 'R2_concat', 'R2_power', 'AICc_exp',
                         'AICc_concat', 'AICc_power', 'bestfitmodel', 'curvature', 'min_peak'])] = [
        results_exp, results_power, results_pow_exp,
        R2_exp, R2_concat, R2_power, AICc_exp,
        AICc_concat, AICc_pow, best_fit_model,
        curvature_model, min_peak
    ]

    print(row.age_cohort_low, row.age_cohort_high, best_fit_model)

/mnt/cube/tsainbur/Projects/github_repos/childes_mi_project/childes_mi/information_theory/model_fitting.py:8: RuntimeWarning: invalid value encountered in log
  return np.abs(np.log(y_true) - np.log(y_model)) * (1 / (np.log(1 + x)))


0.5 1 exp
1.0 1.5 pow_exp
1.5 2 pow_exp
2.0 2.5 pow_exp
2.5 3 pow_exp
3.0 20 pow_exp



In [9]:
MI_DF

,MI,MI_var,shuff_MI,shuff_MI_var,distances,age_cohort_low,age_cohort_high,n_words,n_unique_words,exp_results,...,concat_results,R2_exp,R2_concat,R2_power,AICc_exp,AICc_concat,AICc_power,bestfitmodel,curvature,min_peak
0,"[6.0615547382637684, 5.953026573946322, 5.8999...","[0.06497202939114834, 0.06421181443545038, 0.0...","[5.79799893493386, 5.805771966570329, 5.792474...","[0.06474471004074468, 0.06537742978608413, 0.0...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",0.5,1,2371,72,<lmfit.minimizer.MinimizerResult object at 0x7...,...,<lmfit.minimizer.MinimizerResult object at 0x7...,0.803612,0.804922,0.776286,-136.562178,-132.408084,-127.833763,exp,"[-3.6367098727388944e-06, -5.463370893703798e-...",NaN
1,"[10.373016081368098, 10.130972857078175, 9.987...","[0.00937963690006051, 0.009307961610168364, 0....","[9.60947032947951, 9.60572819037825, 9.5994570...","[0.009097935454051041, 0.009124346562127784, 0...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",1.0,1.5,67747,3609,<lmfit.minimizer.MinimizerResult object at 0x7...,...,<lmfit.minimizer.MinimizerResult object at 0x7...,0.960713,0.986984,0.980082,-420.636087,-526.718807,-488.563146,pow_exp,"[6.528048763553539e-07, 9.767024895467008e-07,...",450.0
2,"[11.349881924265677, 11.087999639009142, 10.96...","[0.005174827403727939, 0.005136394630766223, 0...","[10.681207621740015, 10.677630292519837, 10.67...","[0.005060342080401666, 0.005066649080465537, 0...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",1.5,2,232191,8134,<lmfit.minimizer.MinimizerResult object at 0x7...,...,<lmfit.minimizer.MinimizerResult object at 0x7...,0.949823,0.989080,0.985398,-360.792506,-505.839244,-481.766416,pow_exp,"[9.60855604404184e-07, 1.4398265135829119e-06,...",435.0
3,"[12.421599707857368, 12.124559687616102, 11.99...","[0.0033293605141089407, 0.0032982717697735654,...","[11.756060854089633, 11.754715776334926, 11.75...","[0.003259500059111028, 0.003261010542536753, 0...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",2.0,2.5,656639,14402,<lmfit.minimizer.MinimizerResult object at 0x7...,...,<lmfit.minimizer.MinimizerResult object at 0x7...,0.974186,0.996492,0.991567,-429.445717,-624.641943,-541.319049,pow_exp,"[1.116469991993569e-06, 1.6691921956434772e-06...",375.0
4,"[11.532466104026422, 11.247538669418567, 11.12...","[0.004322162033098881, 0.004289030213930944, 0...","[10.928431808392926, 10.926159127779197, 10.92...","[0.004249227382651435, 0.0042542569785092525, ...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",2.5,3,422922,9336,<lmfit.minimizer.MinimizerResult object at 0x7...,...,<lmfit.minimizer.MinimizerResult object at 0x7...,0.954515,0.985079,0.980663,-349.189828,-455.142679,-433.869505,pow_exp,"[-7.656782712802456e-07, -1.1602581095676295e-...",317.0
5,"[13.90350042312788, 13.60402662450543, 13.4863...","[0.0016126379123154566, 0.001596772378812455, ...","[13.344836093986757, 13.344052969455305, 13.34...","[0.0015852035484411658, 0.001585418309044302, ...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",3.0,20,3210842,43208,<lmfit.minimizer.MinimizerResult object at 0x7...,...,<lmfit.minimizer.MinimizerResult object at 0x7...,0.957765,0.988433,0.983562,-309.589767,-429.516198,-400.182416,pow_exp,"[6.813220961528881e-07, 1.0079629675508996e-06...",335.0


In [10]:
labels = ['6-12 months', '12-18 months', '18-24 months', '24-30 months', '30-36 months', '3-12 years']

### Calculate R2

In [11]:
R2 = MI_DF[['R2_exp', 'R2_concat', 'R2_power']]
R2.columns = ['exp', 'combined', 'power-law']
R2.index = labels
R2 = R2.T
R2

,6-12 months,12-18 months,18-24 months,24-30 months,30-36 months,3-12 years
exp,0.803612,0.960713,0.949823,0.974186,0.954515,0.957765
combined,0.804922,0.986984,0.989080,0.996492,0.985079,0.988433
power-law,0.776286,0.980082,0.985398,0.991567,0.980663,0.983562


### AICc

In [12]:
AICcs = MI_DF[['AICc_exp', 'AICc_concat', 'AICc_power']]
AICcs.columns = ['exp', 'combined.', 'power-law']
AICcs.index = labels
AICcs = AICcs.T
AICcs

,6-12 months,12-18 months,18-24 months,24-30 months,30-36 months,3-12 years
exp,-136.562178,-420.636087,-360.792506,-429.445717,-349.189828,-309.589767
combined.,-132.408084,-526.718807,-505.839244,-624.641943,-455.142679,-429.516198
power-law,-127.833763,-488.563146,-481.766416,-541.319049,-433.869505,-400.182416


#### $\Delta$AIC

In [13]:
delta_AICcs = AICcs.T - np.repeat(np.min(AICcs.T.values, axis=1),3).reshape(len(labels),3)
delta_AICcs = delta_AICcs.T
delta_AICcs

,6-12 months,12-18 months,18-24 months,24-30 months,30-36 months,3-12 years
exp,0.000000,106.082719,145.046738,195.196226,105.952850,119.926431
combined.,4.154093,0.000000,0.000000,0.000000,0.000000,0.000000
power-law,8.728414,38.155660,24.072828,83.322894,21.273174,29.333781


### relative likelihood

In [14]:
relative_likelihoods = mf.relative_likelihood(delta_AICcs)
relative_likelihoods

,6-12 months,12-18 months,18-24 months,24-30 months,30-36 months,3-12 years
exp,1.000000,9.213617e-24,3.187873e-32,4.108450e-43,9.831751e-24,9.084613e-27
combined.,0.125300,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
power-law,0.012725,5.183268e-09,5.924500e-06,8.066098e-19,2.402088e-05,4.268255e-07


### relative probability of models

In [15]:
prob_models = mf.Prob_model_Given_data_and_models(relative_likelihoods)
prob_models

,6-12 months,12-18 months,18-24 months,24-30 months,30-36 months,3-12 years
exp,0.878716,9.213617e-24,3.187854e-32,4.108450e-43,9.831515e-24,9.084609e-27
combined.,0.110103,1.000000e+00,9.999941e-01,1.000000e+00,9.999760e-01,9.999996e-01
power-law,0.011181,5.183268e-09,5.924465e-06,8.066098e-19,2.402031e-05,4.268253e-07


#### Join tables

In [16]:
AICcs['superlabel'] = 'AICc'
relative_likelihoods['superlabel'] = 'Relative likelihood'
R2['superlabel'] = '$r^2$'
prob_models['superlabel'] = 'Relative probability'

In [17]:
results_table = pd.concat([
    AICcs,
    R2,
    relative_likelihoods,
    prob_models
]).round(3).replace(0, '<0.001').replace(1, '>0.999')
results_table[''] = results_table.index
results_table.set_index(['superlabel', ''], inplace=True)
results_table

6-12 months 12-18 months 18-24 months  \
superlabel                                                             
AICc                 exp          -136.562     -420.636     -360.793   
                     combined.    -132.408     -526.719     -505.839   
                     power-law    -127.834     -488.563     -481.766   
$r^2$                exp             0.804        0.961         0.95   
                     combined        0.805        0.987        0.989   
                     power-law       0.776         0.98        0.985   
Relative likelihood  exp            >0.999       <0.001       <0.001   
                     combined.       0.125       >0.999       >0.999   
                     power-law       0.013       <0.001       <0.001   
Relative probability exp             0.879       <0.001       <0.001   
                     combined.        0.11       >0.999       >0.999   
                     power-law       0.011       <0.001       <0.001   

                               24-30 months 30-36 months 3-12 years  
superlabel                                                           
AICc                 exp           -429.446      -349.19    -309.59  
                     combined.     -624.642     -455.143   -429.516  
                     power-law     -541.319      -433.87   -400.182  
$r^2$                exp              0.974        0.955      0.958  
                     combined         0.996        0.985      0.988  
                     power-law        0.992        0.981      0.984  
Relative likelihood  exp             <0.001       <0.001     <0.001  
                     combined.       >0.999       >0.999     >0.999  
                     power-law       <0.001       <0.001     <0.001  
Relative probability exp             <0.001       <0.001     <0.001  
                     combined.       >0.999       >0.999     >0.999  
                     power-law       <0.001       <0.001     <0.001

In [18]:
results_string = results_table.to_latex(bold_rows=True, escape=False)\
      .replace('>', '$>$')\
      .replace('<', '$<$')\
      .replace('superlabel', '')\
     .replace('\n\\textbf', '\n\midrule\n\\textbf')
print(results_string)

\begin{tabular}{llllllll}
\toprule
                     &           & 6-12 months & 12-18 months & 18-24 months & 24-30 months & 30-36 months & 3-12 years \\
\midrule
\textbf{} & {} &             &              &              &              &              &            \\
\midrule
\midrule
\textbf{AICc} & \textbf{exp} &    -136.562 &     -420.636 &     -360.793 &     -429.446 &      -349.19 &    -309.59 \\
                     & \textbf{combined.} &    -132.408 &     -526.719 &     -505.839 &     -624.642 &     -455.143 &   -429.516 \\
                     & \textbf{power-law} &    -127.834 &     -488.563 &     -481.766 &     -541.319 &      -433.87 &   -400.182 \\
\midrule
\textbf{$r^2$} & \textbf{exp} &       0.804 &        0.961 &         0.95 &        0.974 &        0.955 &      0.958 \\
                     & \textbf{combined} &       0.805 &        0.987 &        0.989 &        0.996 &        0.985 &      0.988 \\
                     & \textbf{power-law} &       0.776 &         0